In [1]:
import numpy as np
from math import pi
from scipy.signal import convolve

# References:
# [1] P.P. Vaidyanathan, ROBUST DIGITAL FILTER STRUCTURES, in HANDBOOK FOR DIGITAL SIGNAL PROCESSING. S.K. Mitra and J.F. Kaiser Eds. Wiley-Interscience, N.Y., 1993, Chapter 7.
# [2] P.P. Vaidyanathan, MULTIRATE SYSTEMS AND FILTER BANKS, Prentice Hall, N.Y., Englewood Cliffs, NJ, 1993, Chapter 3.

In [2]:
def numrecursion(r, N, c):
    msg = ''
    q = np.zeros(N + 1, dtype=complex)
    
    # Initialize recursion
    q[0] = np.sqrt(-r[0] / c)
    q[N] = np.conj(c * q[0])
    q[1] = -r[1] / (2 * c * q[0])
    q[N - 1] = np.conj(c * q[1])

    # The limit of the for loop depends on the order being odd or even
    for n in range(3, int(np.ceil(N/2)) + 1):
        q[n-1] = (-r[n-1] / c - np.dot(q[1:n-1], q[n-2:0:-1])) / (2 * q[0])
        q[N+1-n-1] = np.conj(c * q[n-1])
    
    # Compute middle coefficient separately when order is even
    if N % 2 == 0:
        q[int((N+2)/2)-1] = (-r[int((N+2)/2)-1] / c - np.dot(q[1:int((N+2)/2)-1], q[int((N+2)/2)-2:0:-1])) / (2 * q[0])
    
    return q, msg

def powercompnum(b, r, N, *varargin):
    # Compute numerator of power complementary filter.

    msg = ''
    if np.isreal(b).all() and (len(varargin) == 0):
        # Try to get a real q with c = 1 and c = -1
        q = numrecursion(r, N, 1)

        if not np.isreal(q).all():
            q = numrecursion(r, N, -1)

        if not np.isreal(q).all():
            msg = 'A real power complementary filter could not be found'
            print(msg)
            return q
    else:
        # If numerator is complex, we use c=1 when c is not given
        c = 1
        if len(varargin) == 1:
            c = varargin[0]
            if np.max(np.shape(c)) > 1:
                msg = 'C must be a complex scalar.'
                print(msg)
                return None
            if np.abs(c) - 1 > np.finfo(float).eps ** (2/3):
                msg = 'C must be of magnitude one.'
                print(msg)
                return None

        q, msg = numrecursion(r, N, c)
        print(msg)

    return q

def auxpoly(b, a):
    revb = np.conj(b[::-1])
    reva = np.conj(a[::-1])

    r = np.subtract(np.convolve(revb, b), np.convolve(a, reva))
    
    return r

def iirpowcomp(b, a):
    # Find the auxiliary polynomial R(z)
    r = auxpoly(b, a)
    
    # Compute the numerator of the power complementary transfer function
    q, msg = powercompnum(b, r, len(b)-1)
    if msg:
        raise ValueError(msg)

    return q, a

def sortnums(b, bp):
    # Sort numerators prior to calling ALLPASSDECOMPOSITION.
    # ALLPASSDECOMPOSITION always requires the first argument to
    # be symmetric. The second argument can be symmetric or antisymmetric.

    # If b is real and antisymmetric, make it the second argument
    if np.max(np.abs(np.add(b, b[::-1]))) < np.finfo(float).eps ** (2/3):
        p = bp
        q = b
    else:
        p = b
        q = bp
    
    return p, q

def allpassdecomposition(p, q, a):
    # If q is real and antisymmetric, make it imaginary
    if np.all(np.isreal(q)):
        if np.max(np.abs(np.add(q, q[::-1]))) < np.finfo(float).eps ** (2/3):
            q = q * 1j
    
    z = np.roots(p - 1j * q)
    
    # Initialize the allpass functions
    d1 = np.array([1.0])
    d2 = np.array([1.0])

    # Separate the zeros inside the unit circle and the ones outside to form the allpass functions
    for n in range(len(z)):
        if np.abs(z[n]) < 1:
            d2 = convolve(d2, [1, -z[n]])
        else:
            d1 = convolve(d1, [1, -1 / np.conj(z[n])])

    # Remove roundoff imaginary parts
    d1 = imagprune(d1)
    d2 = imagprune(d2)

    beta = np.sum(d2) * (np.sum(p) + 1j * np.sum(q)) / np.sum(a) / np.sum(np.conj(d2))
    
    return d1, d2, beta

def imagprune(poly):
    # Function to remove roundoff imaginary parts
    real_part = np.real(poly)
    imag_part = np.imag(poly)
    imag_part[np.abs(imag_part) < np.finfo(float).eps] = 0
    pruned_poly = real_part + imag_part * 1j
    return pruned_poly

In [3]:
def LinkwitzRiley12dB(fc, sr):
    omega = 2 * pi * fc / sr
    
    sineOmega = np.sin(omega)
    cosineOmega = np.cos(omega)
    a0 = 1 + sineOmega + cosineOmega
    
    A1 = (sineOmega - cosineOmega - 1) / a0
    B0 = (sineOmega) / a0
    B1 = (sineOmega) / a0
    
    b_lp = np.convolve([B0, B1], [B0, B1])
    a_lp = np.convolve([1, A1], [1, A1])
    
    A1 = (sineOmega - cosineOmega - 1) / a0
    B0 = (1 + cosineOmega) / a0
    B1 = -(1 + cosineOmega) / a0
    
    b_hp = np.convolve([B0, B1], [B0, B1])
    a_hp = np.convolve([1, A1], [1, A1])
    
    # two transfer function need to substract to achieve all-pass filter  
    return [-b_lp, a_lp], [b_hp, a_hp]

def LinkwitzRiley24dB(fc, sr):
    omega = 2 * pi * fc / sr
    
    sineOmega = np.sin(omega)
    cosinePmega = np.cos(omega)
    alpha = sineOmega * np.sqrt(2) / 2
    a0 = 1 + alpha
    
    A1 = -( 2 * cosinePmega) / a0
    A2 = (1 - alpha) / a0
    B1 = (1 - cosinePmega) / a0 
    B0 = B1 / 2
    B2 = B0
    
    b_lp = np.convolve([B0, B1, B2], [B0, B1, B2])
    a_lp = np.convolve([1, A1, A2], [1, A1, A2])
    
    A1 = -(2 * cosinePmega) / a0
    A2 = (1 - alpha) / a0
    B1 = -(1 + cosinePmega) / a0
    B0 = -B1 / 2
    B2 = B0
    
    b_hp = np.convolve([B0, B1, B2], [B0, B1, B2])
    a_hp = np.convolve([1, A1, A2], [1, A1, A2])
    
    return [b_lp, a_lp], [b_hp, a_hp]

In [4]:
LP, HP = LinkwitzRiley24dB(60, 48000)

num = np.array([-6.00862921190370605e-08, -1.80258876357111182e-07, -1.80258876357111182e-07, -6.00862921190370605e-08])
den = np.array([1.0, -2.98429207710246303, 2.96870728140036411, -0.984414723607564435])

bp, a = iirpowcomp(num, den)

# p, q = sortnums(num, bp)

# d1, d2, beta = allpassdecomposition(p,q,a)

# print(d1, d2, beta)

# print(LP[0], LP[1])
# print(q, a)
# print(p, q)

C:\Users\zengkuiwen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  
C:\Users\zengkuiwen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
C:\Users\zengkuiwen\AppData\Roaming\Python\Python37\site-packages\numpy\lib\type_check.py:203: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asanyarray(val).imag
